In [1]:
from datasets import load_dataset
import pandas as pd

dataset = load_dataset("divarofficial/real_estate_ads")

In [2]:
# Print the first few examples
print(dataset['train'][:5])

# Get dataset statistics
print(f"Dataset size: {len(dataset['train'])} rows")
print(f"Features: {dataset['train'].features}")


{'cat2_slug': ['temporary-rent', 'residential-sell', 'residential-rent', 'commercial-rent', 'residential-sell'], 'cat3_slug': ['villa', 'apartment-sell', 'apartment-rent', 'office-rent', 'apartment-sell'], 'city_slug': ['karaj', 'tehran', 'tehran', 'tehran', 'mashhad'], 'neighborhood_slug': ['mehrshahr', 'gholhak', 'tohid', 'elahiyeh', 'emamreza'], 'created_at_month': ['2024-08-01 00:00:00', '2024-05-01 00:00:00', '2024-10-01 00:00:00', '2024-06-01 00:00:00', '2024-05-01 00:00:00'], 'user_type': ['مشاور املاک', 'مشاور املاک', None, None, 'مشاور املاک'], 'description': ['۵۰۰متر\n۲۰۰متر بنا دوبلکس\n۳خواب\nاستخر آبگرم داخل\nسیستم صوتی حرفه ای\nسرگرمی ایرهاکی\nبرای اطلاعات بیشتر تماس حاصل فرماید', 'دسترسی عالی به مترو و شریعتی \nمشاعات تمیز \nبدون ایراد \nآخرین بازدید \nفروشنده واقعی\nشایان', 'تخلیه پایان ماه', 'فرشته تاپ لوکیشن\n۹۰ متر موقعیت اداری\nیک اتاق مستر\nدید و وییو مشجر\nروبه باغ\nنور و نقشه عالی\nلابی و لابی من و سرایدار مقیم\nلوکس و شیک\nمدرن\nهمه مشاغل\nبازدید = با قرار داد\nف

Checking for duplicates.

In [ ]:
df = dataset['train'].to_pandas()

# check for duplicates
# print(f"Duplicates before drop: {df.duplicated().sum()}")
# df = df.drop_duplicates()
# print(f"Duplicates after drop: {df.duplicated().sum()}")

# # drop columns if irrelevant
# cols_to_drop = ['title', 'description']  # drop these unless NLP needed
# df = df.drop(columns=cols_to_drop)

# print(df.info())
# print(df.head())

Duplicates before drop: 0
Duplicates after drop: 0


In [4]:
from scipy.stats import pearsonr

features = ['has_pool', 'has_jacuzzi', 'has_sauna', 'has_water', 'has_electricity', 'has_gas']

for feature in features:
    valid_rows = df[[feature, 'price_value']].dropna()
    
    # Convert boolean or object columns to numeric for correlation calculation
    # If these columns are object dtype with 'True'/'False' or similar strings, convert them
    if valid_rows[feature].dtype == 'object':
        valid_rows[feature] = valid_rows[feature].map({'True': 1, 'False': 0})
    elif valid_rows[feature].dtype == 'bool':
        valid_rows[feature] = valid_rows[feature].astype(int)
    
    corr, p_value = pearsonr(valid_rows[feature], valid_rows['price_value'])
    print(f"{feature} -> Correlation: {corr:.6f}, p-value: {p_value:.6f}")

has_pool -> Correlation: nan, p-value: nan
has_jacuzzi -> Correlation: nan, p-value: nan
has_sauna -> Correlation: nan, p-value: nan
has_water -> Correlation: nan, p-value: nan
has_electricity -> Correlation: nan, p-value: nan
has_gas -> Correlation: nan, p-value: nan


In [6]:
luxury_amenities = ['has_pool', 'has_jacuzzi', 'has_sauna']
basic_utilities = ['has_water', 'has_electricity', 'has_gas']

df['has_luxury_amenities'] = df[luxury_amenities].fillna(False).any(axis=1)
df['has_basic_utilities'] = df[basic_utilities].fillna(False).any(axis=1)

# dropping the orginals to reduce sparcity
df.drop(columns=luxury_amenities + basic_utilities, inplace=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 54 columns):
 #   Column                      Non-Null Count    Dtype  
---  ------                      --------------    -----  
 0   cat2_slug                   1000000 non-null  object 
 1   cat3_slug                   999999 non-null   object 
 2   city_slug                   999998 non-null   object 
 3   neighborhood_slug           437139 non-null   object 
 4   created_at_month            1000000 non-null  object 
 5   user_type                   288882 non-null   object 
 6   rent_mode                   352994 non-null   object 
 7   rent_value                  351322 non-null   float64
 8   rent_to_single              19 non-null       float64
 9   rent_type                   103961 non-null   object 
 10  price_mode                  573606 non-null   object 
 11  price_value                 568346 non-null   float64
 12  credit_mode                 352994 non-null   object 
 13

In [7]:
# correlation with price_value (numeric target)
from scipy.stats import pointbiserialr

corr, p_value = pointbiserialr(df['has_luxury_amenities'], df['price_value'].fillna(0))
print(f"Correlation: {corr}, p-value: {p_value}")

Correlation: nan, p-value: nan


/home/arash/.pyenv/versions/3.11.11/lib/python3.11/site-packages/scipy/stats/_stats_py.py:5405: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  rpb, prob = pearsonr(x, y)
